# Example of using LiteQTL

This notebook showcase how to use LiteQTL to perform a QTL scan.

### Dataset: 
We use the BXD Genotype Database (GN Accession: GN600) and a sets of transcriptome data, UTHSCAffy MoGene 1.0 ST Spleen (GN Accession: GN283) from GeneNetwork. 

The genotype file includes 7321 markers by 198 BXD strains; 

The spleen dataset has data for 79 BXD strains and for 35556 transcripts. 

Data cleaning and wrangling was performed using R/qtl (Broman et al. 2003) and R/qtl2 (Broman et al. 2019). 

For details regarding data cleaning, please see [this script](https://github.com/senresearch/LiteQTL-G3-supplement/blob/master/code/clean-data.R)

In [1]:
# Loading packages
using LiteQTL
using DelimitedFiles
using BenchmarkTools

In [9]:
# Location of genotype file and phenotype file
geno_file = joinpath(@__DIR__, "..", "data", "processed", "spleen-bxd-genoprob.csv")
pheno_file = joinpath(@__DIR__, "..", "data","processed", "spleen-pheno-nomissing.csv")

# Whether to output whole LOD matrix or only maximum LOD of each phenotype? Default is false (only maximum)
export_matrix = false

# Location of output 
output_file = "output.csv"

# Setting BLAS threads as 16 threads. 
LiteQTL.set_blas_threads(16);

# Read in data.
G = get_geno_data(geno_file, Float64)
Y = get_pheno_data(pheno_file, Float64, transposed=false)
# getting geno and pheno file size.
n = size(Y,1) # number of individuals
m = size(Y,2) # number of Traits
p = size(G,2) # number of Markers
println("******* Indivuduals n: $n, Traits m: $m, Markers p: $p ****************");

Number of threads using: 16


┌ Info: Removing sex column of phenotype. 
└ @ LiteQTL /export/xiaoqihu/git/LiteQTL.jl/src/data_io.jl:29


******* Indivuduals n: 79, Traits m: 35554, Markers p: 7321 ****************


In [10]:
# run genome scan. 
lod = scan(Y, G,n;export_matrix);
# @benchmark LiteQTL.scan(Y, G,n;export_matrix)

┌ Info: Running genome scan...
└ @ LiteQTL /export/xiaoqihu/git/LiteQTL.jl/src/cpu.jl:91
┌ Info: Done calculating corelation coefficients.
└ @ LiteQTL /export/xiaoqihu/git/LiteQTL.jl/src/cpu.jl:96


Calculating max lod


┌ Info: Done calculating LOD. 
└ @ LiteQTL /export/xiaoqihu/git/LiteQTL.jl/src/cpu.jl:100


35554×2 Array{Float64,2}:
 5046.0   2.03595
 3111.0   2.74552
 6631.0   2.38099
 6632.0   3.03683
 5326.0   2.6874
 5326.0   2.93972
 5325.0   3.40571
 3120.0   2.73382
 3214.0   1.93849
 6631.0   2.45835
 3111.0   2.96902
 5326.0   2.58704
 5326.0   1.90628
    ⋮    
 2975.0   1.69443
 3111.0   4.3085
 2110.0  15.6794
 7255.0   1.97968
 1808.0   2.48822
 6546.0   2.85884
 2110.0   3.24836
 3717.0   2.21924
 7136.0   2.22245
 6647.0   4.70312
 3085.0   2.76619
 3549.0   1.7484

In [4]:
# Uncomment the following lines to run the same function but with GPU: 
# lod = scan(Y, G, n; usegpu=true)
# @benchmark scan(Y, G, n; usegpu=true)

In [5]:
# run genome scan with covariates. 
# X is covariates
# X = ones(size(pheno)[1]) # standard covariates

# Use the first column of phenotype as covariates. 
# X = reshape(Y[:,1], :,1) # make 1 column as a 2d matrix
# Y = Y[:,2:end]

# @time lod = scan(Y, G, X, n; export_matrix)

┌ Info: Running genome scan with covariates...
└ @ LiteQTL /export/xiaoqihu/git/LiteQTL.jl/src/cpu.jl:44


Calculating max lod
 10.645458 seconds (3.74 M allocations: 2.167 GiB, 2.43% gc time)


35553×2 Array{Float64,2}:
 5602.0   2.59794
 5327.0   2.99946
 6680.0   3.28686
 5326.0   2.57237
 5326.0   2.6681
 5325.0   3.04555
 1653.0   2.47555
 5648.0   2.5302
 6631.0   2.32154
 5602.0   2.68334
 5326.0   2.47831
 5326.0   1.89127
 6757.0   1.95527
    ⋮    
 2976.0   1.67575
 3111.0   3.40296
 2110.0  16.3809
 7255.0   2.27785
 1808.0   2.50856
 6547.0   2.80211
 2506.0   3.73299
 2505.0   2.20188
 2939.0   2.3914
 6647.0   4.55006
 3085.0   2.46305
 7283.0   2.35915

In [12]:
# generate plot 
using PlotlyJS
data = histogram(x=lod[:,1],opacity=0.75)
Plot(data)

data: [
  "histogram with fields opacity, type, and x"
]

layout: "layout with field margin"



In [ ]:
# write output to file
writedlm(joinpath(Base.@__DIR__, "..", "data", "results", output_file), lod, ',')

In [13]:
?Plot

search: Plot plot PlotlyJS PlotlyBase PlotlyFrame prevfloat partialsort



```julia
Plot(x, y)
Plot(x, y, l; kind, style, kwargs...)

```

Build a plot of with one trace of type `kind`and set `x` to x and `y` to y. All keyword arguments are passed directly as keyword arguments to the constructed trace.

**NOTE**: If `y` is a matrix, one trace is constructed for each column of `y`

**NOTE**: If `x` and `y` are both matrices, they must have the same number of columns (say `N`). Then `N` traces are constructed, where the `i`th column of `x` is paired with the `i`th column of `y`.

---

```julia
Plot(y)
Plot(y, l; kwargs...)

```

Build a scatter plot and set  `y` to y. All keyword arguments are passed directly as keyword arguments to the constructed scatter.

---

```julia
Plot(f, x0, x1)
Plot(f, x0, x1, l; style, kwargs...)

```

Construct a plot of `f` from `x0` to `x1`, using the layout `l`. All keyword arguments are applied to the constructed trace.

---

```julia
Plot(fs, x0, x1)
Plot(fs, x0, x1, l; style, kwargs...)

```

For each function in `f` in `fs`, construct a scatter trace that plots `f` from `x0` to `x1`, using the layout `l`. All keyword arguments are applied to all constructed traces.

---

```julia
Plot(df)
Plot(df, l; style, kwargs...)

```

Construct a plot using the columns of `df` if possible. For each keyword argument, if the value of the argument is a Symbol and the `df` has a column whose name matches the value, replace the value with the column of the `df`.

If `group` is passed and is a Symbol that is one of the column names of `df`, then call `by(df, group)` and construct one trace per SubDataFrame, passing all other keyword arguments. This means all keyword arguments are passed applied to all traces

---

```julia
Plot(d, x, y)
Plot(d, x, y, l; style, kwargs...)

```

Construct a plot from `df`, passing the provided values of x and y as keyword arguments. See docstring for other method for more information.

---

```julia
Plot(d, y)
Plot(d, y, l; style, kwargs...)

```

Construct a plot from `df`, passing the provided value y as a keyword argument. See docstring for other method for more information.


In [14]:
?plot

search: plot Plot PlotlyJS PlotlyBase PlotlyFrame prevfloat partialsort



No documentation found.

`PlotlyJS.plot` is a `Function`.

```
# 1 method for generic function "plot":
[1] plot(args...; options, kwargs...) in PlotlyJS at /home/xiaoqihu/.julia/packages/PlotlyJS/m2Lzd/src/display.jl:144
```
